## <center>Сложные соединения<center>

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим. Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](images/relations.jpg)

#### **Задание 1.1**

Укажите название города с максимальным весом единичной доставки.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**NATURAL JOIN** sql.city c  
**GROUP BY**  
&ensp;&ensp;&ensp;&ensp;c.city_id,  
&ensp;&ensp;&ensp;&ensp;s.weight  
**HAVING** s.weight = **MAX**(s.weight)  
**ORDER BY** s.weight **DESC**  
**LIMIT** 1

Green Bay

Сколько различных производителей грузовиков перечислено в таблице truck?

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;**COUNT**(**DISTINCT** t.make)  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.truck t

3

Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;d.first_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.driver d **ON** s.driver_id = d .driver_id  
**GROUP BY** d.driver_id, s.cust_id  
**ORDER BY COUNT**(s.ship_id) **DESC**  
**LIMIT** 1

Holger

Укажите дату первой по времени доставок в таблице shipment.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;s.ship_date  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
**ORDER BY** s.ship_date  
**LIMIT** 1  

08.01.2016

Укажите дату последней по времени доставок в таблице shipment.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;s.ship_date  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
**ORDER BY** s.ship_date **DESC**  
**LIMIT** 1

27.12.2017

Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.cust_name  
**FROM**  
&ensp;&ensp;&ensp;&ensp;sql.shipment s  
&ensp;&ensp;&ensp;&ensp;**JOIN** sql.customer c **ON* s.cust_id = c.cust_id  
**GROUP BY** c.cust_id  
**ORDER BY COUNT**(s.ship_id) **DESC**
**LIMIT** 1

Autoware Inc

### <center>Принцип и условия работы UNION<center>

Мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм. Для этого напишем простой запрос:

**SELECT**  
&ensp;&ensp;&ensp;&ensp;book_name object_name,  
&ensp;&ensp;&ensp;&ensp;'книга' object_description  
**FROM** public.books  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;movie_title,   
&ensp;&ensp;&ensp;&ensp;'фильм'   
**FROM** sql.kinopoisk

Визуально произведённое нами действие можно представить следующим образом:

![](images/union_all.jpg)

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них **одинаковая структура**, а именно:

* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

#### <center>Виды UNION<center>

Оператор присоединения существует в двух вариантах:

**UNION** выводит только уникальные записи;  
**UNION ALL** присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности. 

Важно! **UNION** оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с **DISTINCT**). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать **UNION ALL**.

#### <center>Ситнаксис<center>

Запрос строится таким образом:

**SELECT** n columns  
**FROM**   
&ensp;&ensp;&ensp;&ensp;table_1  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**  
&ensp;&ensp;&ensp;&ensp;table_2  
...  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;n columns  
**FROM**   
&ensp;&ensp;&ensp;&ensp;table_n    

Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.  
**Важно!** Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.


Сортировку любой части с использованием ORDER BY и LIMIT можно вести любой части запроса, обернув ее в круглые скобки.

(**SELECT** book_name object_name, 'книга' object_descritption  
**FROM** public.books  
**ORDER BY** 1  
**LIMIT** 1)  
**UNION ALL**  
(**SELECT** movie_title, 'фильм'   
**FROM** sql.kinopoisk  
**ORDER BY** 1  
**LIMIT** 1)

#### **Задание 2.1**

Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;**DISTINCT** city_name object_name, 'city' object_type  
**FROM** sql.city  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;**DISTINCT** state, 'state'  
**FROM** sql.city  
**UNION ALL**  
**SELECT**
&ensp;&ensp;&ensp;&ensp;first_name, 'driver'  
**FROM** sql.driver  
**UNION ALL**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;**DISTINCT** make, 'truck'  
**FROM** sql.truck  
**ORDER BY** object_name, object_type

#### **Задание 2.2**

Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

*Запрос:*

**SELECT** 
&ensp;&ensp;&ensp;&ensp;city_name object_name  
**FROM** sql.city   
**UNION ALL**    
**SELECT**  
    &ensp;&ensp;&ensp;&ensp;state  
**FROM** sql.city  
**ORDER BY** object_name

#### **Задание 2.3**

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.  

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;city_name object_name  
**FROM** sql.city    
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;state  
**FROM** sql.city  
**ORDER BY** object_name

### <center>Ограничение типов данных<center>

**Важно!** Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;c.city_id::text  
**FROM** sql.city c  
**UNION ALL**    
**SELECT**  
&ensp;&ensp;&ensp;&ensp;cc.city_name  
**FROM** sql.city cc

#### **Задание 3.1**

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.  

*Запрос:*

**SELECT**  
&ensp;&ensp;&ensp;&ensp;zip_code::text contact,   
&ensp;&ensp;&ensp;&ensp;first_name first_name,   
&ensp;&ensp;&ensp;&ensp;'zip' contact_type  
**FROM** sql.driver  
**UNION**  
**SELECT**  
&ensp;&ensp;&ensp;&ensp;phone::text contact,   
&ensp;&ensp;&ensp;&ensp;first_name,  
&ensp;&ensp;&ensp;&ensp;'phone'  
**FROM** sql.driver  
**ORDER BY** contact, first_name  